# Regression

## Classic regression solution using Scikit-Learn

Let's use the California housing dataset. You can get it from Scikit-Learn library. Description of the dataset is available [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html#sklearn.datasets.fetch_california_housing)

In [1]:
# Dirty patch to fix SSL error
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

In [11]:
# Import the California housing dataset
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

In [3]:
import pandas as pd

housing_df = pd.DataFrame(housing.data, columns=housing.feature_names)
housing_df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [4]:
housing_df['target'] = housing.target
housing_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


What if Ridge wasn't working?

We could always try another model, 
    
How about to try ensemble model (an ensemble model is a model made up of many models)

In [5]:
# Let's try the Random Forest Regressor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Setup random seed
np.random.seed(42)

# Create the data
X = housing_df.drop("target", axis=1)
y = housing_df["target"]

# split into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Instantiate Random Forest Regressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

# Check the score of the Random Forest Regressor model on test data
rf.score(X_test, y_test)

0.8059809073051385

In [12]:
# let's predict the first 10 rows of the test data
rf.predict(X_test[:10])

array([0.49058  , 0.75989  , 4.9350165, 2.55864  , 2.33461  , 1.6580801,
       2.34237  , 1.66708  , 2.5609601, 4.8519781])

In [13]:
# Compare the predictions to the actual values
np.array(y_test[:10])

array([0.477  , 0.458  , 5.00001, 2.186  , 2.78   , 1.587  , 1.982  ,
       1.575  , 3.4    , 4.466  ])

# Let's try to do the sa,e with OpenAI

let's prepare a dataset for OpenAI model to train

We will use the same dataset as above, but we will split it into training and test datasets

In [74]:
# get first 1000 rows for training
housing_df_train = housing_df[:1000].copy()

# get the 20 of the rest for testing
housing_df_test = housing_df[1000:1020].copy()

In [75]:
# save the training dataset to csv file
housing_df_train.to_csv('data/regression/housing_train.csv', index=False)

# remove the target column from the test dataset
targets = housing_df_test['target']

# fill the target column with zeros
housing_df_test["target"] = 0

# save the test dataset to csv file
housing_df_test.to_csv('data/regression/housing_test.csv', index=False)

In [56]:
# check the data
# housing_df_train.head()
# housing_df_test

In [18]:
# Load OpenAI
import dotenv
from openai import OpenAI

dotenv.load_dotenv()

client = OpenAI()

In [20]:
# upload the file to OpenAI
file = client.files.create(file=open("data/regression/housing_train.csv", "rb"), purpose="assistants")

In [68]:
# create an assistant
assistant = client.beta.assistants.create(
  instructions="You are a linear regression tool. "
               "You have a training dataset file with the California housing dataset. It contains 1000 rows and 9 columns. "
               "The columns are: MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude, target. "
                "The target column is the target variable. "
                "Your task is to analyze the data and provide the best possible linear regression model to predict the target variable for the file, provided by user in the next message. "
               "Tou have to return the json object with the same structure as the input file, but with the target column filled with the predictions instead of zeros"
               "Please respond only in json_object format. Do not add any additional information above or below.",
  model="gpt-4o-mini",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }
)

In [63]:
# upload the test file to OpenAI
file_test = client.files.create(file=open("data/regression/housing_test.csv", "rb"), purpose="assistants")

In [69]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Read the provided CSV file with California housing data without a terget column. Make a target column prediction based on the training dataset",
      "attachments": [
        {
          "file_id": file_test.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ],
    }
  ]
)

In [70]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, 
    assistant_id=assistant.id,
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

In [71]:
messages[0].content[0].text.value

'[{"MedInc":3.3841,"HouseAge":29.0,"AveRooms":4.8420310296,"AveBedrms":1.0028208745,"Population":1919.0,"AveOccup":2.706629055,"Latitude":37.69,"Longitude":-121.76,"target":2.1294933233},{"MedInc":3.5532,"HouseAge":36.0,"AveRooms":4.5349462366,"AveBedrms":1.0,"Population":950.0,"AveOccup":2.5537634409,"Latitude":37.68,"Longitude":-121.77,"target":2.103695646},{"MedInc":3.3618,"HouseAge":34.0,"AveRooms":5.114967462,"AveBedrms":1.0802603037,"Population":1157.0,"AveOccup":2.5097613883,"Latitude":37.69,"Longitude":-121.78,"target":2.063115922},{"MedInc":4.3533,"HouseAge":35.0,"AveRooms":4.9877622378,"AveBedrms":1.027972028,"Population":1761.0,"AveOccup":3.0786713287,"Latitude":37.69,"Longitude":-121.78,"target":2.4223743787},{"MedInc":4.5357,"HouseAge":25.0,"AveRooms":6.0133715377,"AveBedrms":1.0334288443,"Population":3200.0,"AveOccup":3.0563514804,"Latitude":37.69,"Longitude":-121.79,"target":2.6087812997},{"MedInc":5.7068,"HouseAge":9.0,"AveRooms":5.8615611193,"AveBedrms":1.0839469809,"P

In [72]:
import json

response = messages[0].content[0].text.value
response = response.replace("```json\n", "").replace("```", "")

# Parse the response
try:
    predictions = json.loads(response)
except json.JSONDecodeError:
    clusters = {}
    print("Failed to parse JSON response.")


predictions

[{'MedInc': 3.3841,
  'HouseAge': 29.0,
  'AveRooms': 4.8420310296,
  'AveBedrms': 1.0028208745,
  'Population': 1919.0,
  'AveOccup': 2.706629055,
  'Latitude': 37.69,
  'Longitude': -121.76,
  'target': 2.1294933233},
 {'MedInc': 3.5532,
  'HouseAge': 36.0,
  'AveRooms': 4.5349462366,
  'AveBedrms': 1.0,
  'Population': 950.0,
  'AveOccup': 2.5537634409,
  'Latitude': 37.68,
  'Longitude': -121.77,
  'target': 2.103695646},
 {'MedInc': 3.3618,
  'HouseAge': 34.0,
  'AveRooms': 5.114967462,
  'AveBedrms': 1.0802603037,
  'Population': 1157.0,
  'AveOccup': 2.5097613883,
  'Latitude': 37.69,
  'Longitude': -121.78,
  'target': 2.063115922},
 {'MedInc': 4.3533,
  'HouseAge': 35.0,
  'AveRooms': 4.9877622378,
  'AveBedrms': 1.027972028,
  'Population': 1761.0,
  'AveOccup': 3.0786713287,
  'Latitude': 37.69,
  'Longitude': -121.78,
  'target': 2.4223743787},
 {'MedInc': 4.5357,
  'HouseAge': 25.0,
  'AveRooms': 6.0133715377,
  'AveBedrms': 1.0334288443,
  'Population': 3200.0,
  'AveOccu

In [79]:
# output the predictions target column
predicted_targets = [row['target'] for row in predictions]
predicted_targets

[2.1294933233,
 2.103695646,
 2.063115922,
 2.4223743787,
 2.6087812997,
 3.0317640844,
 2.2129483562,
 2.7037047523,
 2.2657313518,
 1.8837323336,
 3.541059233,
 1.9896380601,
 1.7053532834,
 2.2251972238,
 2.5544119146,
 3.1225371793,
 3.1536421104,
 2.6224742618,
 2.9940670871,
 2.4784510802]

In [76]:
targets

1000    1.844
1001    1.584
1002    1.746
1003    1.684
1004    1.884
1005    2.567
1006    1.838
1007    1.834
1008    1.775
1009    1.869
1010    2.389
1011    1.725
1012    1.792
1013    2.044
1014    2.094
1015    2.642
1016    3.500
1017    1.998
1018    2.110
1019    2.011
Name: target, dtype: float64

## Summary

This is not a perfect solution to use OpenAI for regression tasks